In [ ]:
import torch
import torch.nn as nn
# hyper parameters
num_epochs = 5000
learning_rate = 1e-2
early_stopping_rounds = 5

# linear regression model
class LinearRegression(nn.Module):

    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out
    
    def train(self, X_train, y_train):
        y_train = np.expand_dims(y_train, axis=1)
        inputs = torch.from_numpy(X_train).float()
        targets = torch.from_numpy(y_train).float()

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = torch.sqrt(criterion(outputs, targets))
        loss.backward()
        optimizer.step()

        return loss.item()

    def valid(self, X_val, y_val):
        y_val = np.expand_dims(y_val, axis=1)
        inputs = torch.from_numpy(X_val).float()
        targets = torch.from_numpy(y_val).float()

        outputs = model(inputs)
        val_loss = criterion(outputs, targets)

        return val_loss.item()

model = LinearRegression(input_size=len(use_cols), output_size=1)

# loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#========================================================================
# Result Box
model_list = []
result_list = []
score_list = []
val_pred_list = []
test_pred = np.zeros(len(test))
x_test = x_test.astype(np.float32)
#========================================================================

#========================================================================
# Train & Prediction Start

for fold_no, (trn_idx, val_idx) in enumerate(zip(*kfold)):

    #========================================================================
    # Make Dataset
    
    X_train, y_train = train.loc[train[key].isin(trn_idx), :][use_cols], Y.loc[train[key].isin(trn_idx)].values
    X_val, y_val = train.loc[train[key].isin(val_idx), :][use_cols], Y.loc[train[key].isin(val_idx)].values
    
    X_train[:] = scaler.transform(X_train)
    X_val[:] = scaler.transform(X_val)
    X_train = X_train.as_matrix().astype(np.float32)
    X_val = X_val.as_matrix().astype(np.float32)
    
    #========================================================================
    # Fitting
    loss_list = []
    val_loss_list = []
    best_loss = 10**10
    early_stop_cnt = 0
    for epoch in range(num_epochs):

        loss = model.train(X_train, y_train)
        val_loss = model.valid(X_val, y_val)
        if best_loss>val_loss:
            best_loss = val_loss
        else:
            early_stop_cnt+=1

        if epoch % 200 == 0:
            print('epoch %d, loss: %.4f val_loss: %.4f' % (epoch, loss, val_loss))

        loss_list.append(loss)
        val_loss_list.append(val_loss)
        if early_stop_cnt>early_stopping_rounds:
            break
    #========================================================================
    
    # Prediction
    if is_oof:
        val_id_list = list(set(train_ids) - set(trn_idx))
        X_val = train.loc[train[key].isin(val_id_list), :][use_cols]
        y_val = Y.loc[train[key].isin(val_id_list)]
        y_pred = model(torch.from_numpy(X_val)).detach().numpy()
        tmp_val = train.loc[train[key].isin(val_id_list), :][[key, target]].set_index(key)
        tmp_val['prediction'] = y_pred
        df_stack['pred_{fold_no}'] = tmp_val['prediction']
        del tmp_val
        gc.collect()
    else:
        y_pred = model(torch.from_numpy(X_val)).detach().numpy()
    out_cnt = (y_pred==np.inf).sum()+(y_pred==-np.inf).sum()+(y_pred!=y_pred).sum()

    if out_cnt>0:
        print("Exist Inf or NaN")
        sys.exit()
    
    y_pred = y_pred.reshape(y_pred.shape[0], )
    tmp_pred = model(torch.from_numpy(x_test)).detach().numpy()
    test_pred += tmp_pred.reshape(tmp_pred.shape[0], )
    test['prediction'] = test_pred
    test = test[[key, 'prediction']]
    
    if is_plus:
        y_val += (y_min-1)
        y_pred += (y_min-1)
        test_pred += (y_min-1)
#     model_list.append(model)
    
    # Stack Prediction
    df_pred = train.loc[train[key].isin(val_idx), :][[key, target]].copy()
    df_pred['prediction'] = y_pred
    result_list.append(df_pred)
    
    # Scoring
    if out_part=='clf':
        score = roc_auc_score(y_val, y_pred)
        print(f'RMSE: {score} | SUM ERROR: {err.sum()}')
    else:
        err = (y_val - y_pred)
        score = np.sqrt(mean_squared_error(y_val, y_pred))
        print(f'RMSE: {score} | SUM ERROR: {err.sum()}')
    score_list.append(score)
    #========================================================================

cv_score = np.mean(score_list)
logger.info(f'''
#========================================================================
# CV SCORE AVG: {cv_score}
#========================================================================''')

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F


# hyper parameters
learning_rate = 1e-2
epochs = 200
early_stopping_rounds = 50
best_loss = 10**10
early_stop_cnt = 0

# Building model
class Model(nn.Module):
    def __init__(self, input_cols, hidden, output):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input_cols, 192)
        self.l2 = nn.Linear(192 , 128)
        self.l3 = nn.Linear(128 , 64)
        self.l4 = nn.Linear(64 , 32)
        self.l5 = nn.Linear(32 , 16)
        self.l6 = nn.Linear(16, output)
    
    def forward(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        out = F.relu(self.l4(out))
        out = F.relu(self.l5(out))
        out = self.l6(out)
        return out        
    
    
# Trainig
model = Model(input_cols=len(use_cols), hidden=64, output=2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


def print_(loss):
    print ("The loss calculated: ", loss)
    
    # Make Dataset
X_train, y_train = train.loc[train[key].isin(trn_idx), :][use_cols], Y.loc[train[key].isin(trn_idx)].values
X_val, y_val = train.loc[train[key].isin(val_idx), :][use_cols], Y.loc[train[key].isin(val_idx)].values

X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_train = X_train.as_matrix()
X_val = X_val.as_matrix()
    
# Not using dataloader
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
for epoch in range(1, epochs+1):
    if epoch%10==0:
        print ("Epoch #",epoch)
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    
    if best_loss>loss.item():
        best_loss = loss
    else:
        early_stop_cnt+=1
        
    print_(loss.item())
    if early_stop_cnt>early_stopping_rounds:
        break
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward() # Gradients
    optimizer.step() # Update
    
# Prediction
X_val = Variable(torch.from_numpy(X_val)).float()
y_pred = model(X_val)
y_pred = torch.max(y_pred,1)[1]
# y_pred = y_pred.detach().numpy()
score = roc_auc_score(y_val, y_pred)
print(f'AUC: {score}')